# COUNT,集計関数,WINDOW関数
### 練習項目
- pass

### 参考文献
***
- [SQL第2版ゼロからはじめるデータベース操作](https://www.shoeisha.co.jp/book/detail/9784798144450)
- [ビッグデータ分析・活用のためのSQLレシピ](https://book.mynavi.jp/supportsite/detail/9784839961268.html)

In [1]:
%reload_ext google.cloud.bigquery

In [2]:
import pandas as pd

## テーブル定義

In [32]:
p_id = 'tactile-acrobat-231812'
name = 'example.review'
data = [
    ('U001', 'A001', 94),
    ('U001', 'A002', 81),
    ('U001', 'A003', 78),
    ('U002', 'A001', 64),
    ('U002', 'A002', 90),
    ('U002', 'A003', 82),
    ('U003', 'A001', 78),
    ('U003', 'A002', 58),
    ('U003', 'A003', 55),
]
cols = ['user_id', 'product_id', 'score']

### CREATE

In [33]:
df_create = pd.DataFrame(data, columns=cols)
df_create.to_gbq(name, p_id)

1it [00:04,  4.87s/it]


### UPDATE

In [29]:
%%bigquery
UPDATE `tactile-acrobat-231812.example.review` 
    set amount=100
WHERE
    dt = 'A'

""


### INSERT

In [42]:
%%bigquery
INSERT INTO `tactile-acrobat-231812.example.review` 
    (dt, amount, agg_amount, year_avg_amount)
VALUES
    ('A',0,1,2),
    ('B',1,2,3)

""


### DELETE

In [39]:
%%bigquery
DELETE FROM `tactile-acrobat-231812.example.review`
WHERE
    dt = '2015-01'

""


### DROP TABLE

In [29]:
%%bigquery
DROP TABLE `tactile-acrobat-231812.example.review`

""


### SELECT（*）

In [7]:
%%bigquery df_ast
SELECT
  *
FROM 
  `tactile-acrobat-231812.example.review`

In [8]:
display(df_ast)

,user_id,product_id,score
0,U001,A001,4.0
1,U001,A002,5.0
2,U001,A003,5.0
3,U002,A001,3.0
4,U002,A002,3.0
5,U002,A003,4.0
6,U003,A001,5.0
7,U003,A002,4.0
8,U003,A003,4.0


### 全体レコード集計

In [34]:
%%bigquery df_all_agg
SELECT
    COUNT(*) AS total_count
    ,COUNT(DISTINCT user_id) AS user_count
    ,COUNT(DISTINCT product_id) AS product_count
    ,SUM(score) AS score_sum
    ,AVG(score) AS score_avg
    ,VARIANCE(score) AS score_var
    ,STDDEV(score) AS score_std
    ,MIN(score) AS score_min
    ,MAX(score) AS score_max
FROM 
    `tactile-acrobat-231812.example.review`

In [35]:
display(df_all_agg)

,total_count,user_count,product_count,score_sum,score_avg,score_var,score_std,score_min,score_max
0,9,3,3,680,75.555556,187.027778,13.67581,55,94


### user_id別レコード集計

In [36]:
%%bigquery df_user_id_agg
SELECT
    user_id
    ,COUNT(*) AS total_count　-- GROUP BYでuser_idを指定しているためuser_idごとの全体カウントとなる
    ,COUNT(DISTINCT product_id) AS product_count
    ,SUM(score) AS score_sum
    ,AVG(score) AS score_avg
    ,VARIANCE(score) AS score_var
    ,STDDEV(score) AS score_std
    ,MIN(score) AS score_min
    ,MAX(score) AS score_max
FROM 
    `tactile-acrobat-231812.example.review`
GROUP BY
    user_id -- user_idごとにテーブルを分けることをイメージ

In [37]:
display(df_user_id_agg)

,user_id,total_count,product_count,score_sum,score_avg,score_var,score_std,score_min,score_max
0,U003,3,3,191,63.666667,156.333333,12.503333,55,78
1,U002,3,3,236,78.666667,177.333333,13.316656,64,90
2,U001,3,3,253,84.333333,72.333333,8.504901,78,94


### user_id別レコード集計（元テーブル残すver）

In [38]:
%%bigquery df_user_id_agg_over
SELECT
    user_id
    ,product_id
    ,score
    ,AVG(score) OVER() AS score_avg
    ,AVG(score) OVER(PARTITION BY user_id) AS score_avg_per_user
    ,score - AVG(score) OVER(PARTITION BY user_id) AS score_diff_all_per_user
FROM 
    `tactile-acrobat-231812.example.review`

In [39]:
display(df_user_id_agg_over)

,user_id,product_id,score,score_avg,score_avg_per_user,score_diff_all_per_user
0,U001,A001,94,75.555556,84.333333,9.666667
1,U001,A002,81,75.555556,84.333333,-3.333333
2,U001,A003,78,75.555556,84.333333,-6.333333
3,U002,A001,64,75.555556,78.666667,-14.666667
4,U002,A002,90,75.555556,78.666667,11.333333
5,U002,A003,82,75.555556,78.666667,3.333333
6,U003,A001,78,75.555556,63.666667,14.333333
7,U003,A002,58,75.555556,63.666667,-5.666667
8,U003,A003,55,75.555556,63.666667,-8.666667


### 順序に関連する関数

In [45]:
%%bigquery df_user_id_orderby
SELECT
    user_id
    ,score
    ,ROW_NUMBER() OVER(ORDER BY score DESC) AS score_row -- 一意のランキング
    ,RANK() OVER(ORDER BY score DESC) AS score_rank -- タイ込のランキング
    ,DENSE_RANK() OVER(ORDER BY score DESC) AS score_dense -- タイ込のランキング（順位飛ばさない）
    ,LAG(user_id) OVER(ORDER BY score DESC) AS lag1
    ,LAG(user_id, 2) OVER(ORDER BY score DESC) AS lag2
    ,LEAD(user_id) OVER(ORDER BY score DESC) AS lead1
    ,LEAD(user_id, 2) OVER(ORDER BY score DESC) AS lead2
FROM 
    `tactile-acrobat-231812.example.review`
-- 全体をscore_rowによって並べ替える
ORDER BY score_row

In [46]:
display(df_user_id_orderby)

,user_id,score,score_row,score_rank,score_dense,lag1,lag2,lead1,lead2
0,U001,94,1,1,1,None,None,U002,U002
1,U002,90,2,2,2,U001,None,U002,U001
2,U002,82,3,3,3,U002,U001,U001,U001
3,U001,81,4,4,4,U002,U002,U001,U003
4,U001,78,5,5,5,U001,U002,U003,U002
5,U003,78,6,5,5,U001,U001,U002,U003
6,U002,64,7,7,6,U003,U001,U003,U003
7,U003,58,8,8,7,U002,U003,U003,None
8,U003,55,9,9,8,U003,U002,None,None
